## Imports

In [4]:
import pickle
import os
import ast
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import diags, eye
from sklearn.svm import SVC
from scipy.signal import find_peaks, savgol_filter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from scipy.sparse.linalg import spsolve
from sklearn.model_selection import train_test_split
from pyHSICLasso import HSICLasso

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import warnings
import matplotlib
matplotlib.use('Agg')  # Use the Agg backend which does not require a windowing system
import matplotlib.pyplot as plt


# Ignore all warnings
warnings.filterwarnings('ignore')

c:\Users\Horiz\anaconda3\lib\site-packages\dask\dataframe\utils.py:365: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\Horiz\anaconda3\lib\site-packages\dask\dataframe\utils.py:365: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
c:\Users\Horiz\anaconda3\lib\site-packages\dask\dataframe\utils.py:365: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [7]:
lectura = pd.read_csv('brca_data_w_subtypes.csv')
lectura = lectura.drop(columns=['Unnamed: 0'])
lectura

,rs_CLEC3A,rs_CPB1,rs_SCGB2A2,rs_SCGB1D2,rs_TFF1,rs_MUCL1,rs_GSTM1,rs_PIP,rs_ADIPOQ,rs_ADH1B,...,pp_p38.MAPK,pp_p38.pT180.Y182,pp_p53,pp_p62.LCK.ligand,pp_p70S6K,pp_p70S6K.pT389,pp_p90RSK,pp_p90RSK.pT359.S363,vital.status,histological.type
0,0.892818,6.580103,14.123672,10.606501,13.189237,6.649466,10.520335,10.338490,10.248379,10.229970,...,-0.002598,0.449228,-0.375230,-0.691766,-0.337863,-0.178503,0.011638,-0.207257,0,infiltrating ductal carcinoma
1,0.000000,3.691311,17.116090,15.517231,9.867616,9.691667,8.179522,7.911723,1.289598,1.818891,...,0.220809,1.035115,-0.074136,0.279067,0.292925,-0.155242,-0.089365,0.267530,0,infiltrating ductal carcinoma
2,3.748150,4.375255,9.658123,5.326983,12.109539,11.644307,10.517330,5.114925,11.975349,11.911437,...,-0.133214,0.344969,-0.351936,0.219910,0.308110,-0.190794,-0.222150,-0.198518,0,infiltrating ductal carcinoma
3,0.000000,18.235519,18.535480,14.533584,14.078992,8.913760,10.557465,13.304434,8.205059,9.211476,...,-0.384008,0.678042,0.096329,-0.266554,-0.079871,-0.463237,0.522998,-0.046902,0,infiltrating ductal carcinoma
4,0.000000,4.583724,15.711865,12.804521,8.881669,8.430028,12.964607,6.806517,4.294341,5.385714,...,0.209858,0.920408,0.042210,-0.441542,-0.152317,0.511386,-0.096482,0.037473,0,infiltrating ductal carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,0.000000,5.004281,12.817877,10.854036,11.293350,4.143573,0.926986,5.818401,12.950524,12.552395,...,-0.103128,0.326742,-2.627649,-0.993249,0.131803,-0.012965,0.306601,0.344012,0,infiltrating lobular carcinoma
701,2.241901,4.867086,8.875779,5.641164,11.940968,3.149715,4.101322,9.272752,5.335362,6.974459,...,-0.144137,0.293718,-0.683490,-0.502791,-0.674763,0.165243,-0.338279,0.487583,1,infiltrating lobular carcinoma
702,3.260718,6.700652,14.299072,8.443970,12.312343,11.846810,2.148544,10.435503,3.498442,8.011837,...,0.030552,-0.771497,0.217850,0.669064,0.110822,-0.170345,-0.232004,-0.165477,1,infiltrating lobular carcinoma
703,11.766777,6.656791,13.638154,10.618453,13.775750,6.649667,7.340151,9.903519,9.922855,9.360980,...,0.651704,-0.136779,-0.126707,0.053346,-0.455617,-0.219365,-0.103807,-0.109356,0,infiltrating lobular carcinoma


In [8]:

data_rs = lectura.filter(regex='^rs|vital.status|histological.type')
data_rs


,rs_CLEC3A,rs_CPB1,rs_SCGB2A2,rs_SCGB1D2,rs_TFF1,rs_MUCL1,rs_GSTM1,rs_PIP,rs_ADIPOQ,rs_ADH1B,...,rs_ACVR1C,rs_AMY1A,rs_OXTR,rs_DEFB132,rs_APOB,rs_SPHKAP,rs_DPYSL5,rs_HEPN1,vital.status,histological.type
0,0.892818,6.580103,14.123672,10.606501,13.189237,6.649466,10.520335,10.338490,10.248379,10.229970,...,5.012278,0.514400,9.080063,5.321376,0.892818,1.651683,0.000000,2.860625,0,infiltrating ductal carcinoma
1,0.000000,3.691311,17.116090,15.517231,9.867616,9.691667,8.179522,7.911723,1.289598,1.818891,...,2.314029,0.000000,5.505313,2.509696,0.444879,3.769803,0.000000,0.000000,0,infiltrating ductal carcinoma
2,3.748150,4.375255,9.658123,5.326983,12.109539,11.644307,10.517330,5.114925,11.975349,11.911437,...,6.805251,1.500292,6.335281,4.296876,3.708331,0.000000,0.000000,4.554828,0,infiltrating ductal carcinoma
3,0.000000,18.235519,18.535480,14.533584,14.078992,8.913760,10.557465,13.304434,8.205059,9.211476,...,4.484248,1.774249,7.223634,0.000000,1.593163,0.853517,0.000000,0.000000,0,infiltrating ductal carcinoma
4,0.000000,4.583724,15.711865,12.804521,8.881669,8.430028,12.964607,6.806517,4.294341,5.385714,...,2.710790,1.575796,6.382129,3.092157,0.000000,0.841893,0.000000,1.755828,0,infiltrating ductal carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,0.000000,5.004281,12.817877,10.854036,11.293350,4.143573,0.926986,5.818401,12.950524,12.552395,...,7.447096,1.486766,6.128252,6.552048,4.762237,0.000000,0.000000,6.565015,0,infiltrating lobular carcinoma
701,2.241901,4.867086,8.875779,5.641164,11.940968,3.149715,4.101322,9.272752,5.335362,6.974459,...,2.578601,1.801904,6.931018,3.149715,1.410287,0.870976,0.000000,0.000000,1,infiltrating lobular carcinoma
702,3.260718,6.700652,14.299072,8.443970,12.312343,11.846810,2.148544,10.435503,3.498442,8.011837,...,1.193898,3.194623,8.584488,4.002081,0.515208,0.894139,1.193898,1.441961,1,infiltrating lobular carcinoma
703,11.766777,6.656791,13.638154,10.618453,13.775750,6.649667,7.340151,9.903519,9.922855,9.360980,...,4.546327,0.000000,6.276884,1.801200,0.582074,0.000000,0.000000,2.983404,0,infiltrating lobular carcinoma


## Inicializar Weights and Biases

In [9]:
wandb.init(#entity = 'algo' para que se vaya guardando todo en el mismo orden conforme se van subiendo
    project = "practice2_salud"
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diegocollado (practica_maldi). Use `wandb login --relogin` to force relogin


# Preprocesamiento de datos

In [3]:
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    data["MALDI_binned"] = data["MALDI_binned"].apply(ast.literal_eval)
    return data

data = load_and_preprocess_data("practica_micro.csv")

## Análisis de desbalanceo de datos

In [4]:
def count_samples(data):
    n_ery = data[data["Erythromycin"] == 1.0].shape[0]
    n_cip = data[data["Ciprofloxacin"] == 1.0].shape[0]
    n_cip_ery = data[(data["Erythromycin"] == 1.0) & (data["Ciprofloxacin"] == 1.0)].shape[0]
    no_res = data[(data["Erythromycin"] == 0.0) & (data["Ciprofloxacin"] == 0.0)].shape[0]
    return n_ery, n_cip, n_cip_ery, no_res

n_ery, n_cip, n_cip_ery, no_res = count_samples(data)
print(f"Number of samples with erythromycin resistance: {n_ery}")
print(f"Number of samples with ciprofloxacin resistance: {n_cip}")
print(f"Number of samples with both resistances: {n_cip_ery}")
print(f"Number of samples with no resistance: {no_res}")


Number of samples with erythromycin resistance: 344
Number of samples with ciprofloxacin resistance: 117
Number of samples with both resistances: 57
Number of samples with no resistance: 1327


Labels
- 0: No resistente
- 1: Resistente a Ciprofloxacino
- 2: Resistente a Eritromicina
- 3: Resistente a ambos

## Preprocesamiento de datos
- Separación de datos en entrenamiento y test
- Etiquetado
- Baseline Correction using Asymmetric Least Squares (ALS)
- Savitzky-Golay Filter
- Normalización de datos
- Detección de picos
- Expansión de columnas del dataframe

In [5]:
def prepare_data_for_model(data, test_size=0.2):
    
    X = data.drop(columns=['vital.status'])
        
    
    X_train, X_test, y_train, y_test = train_test_split(data, data['vital.status'], test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test



X_train, X_test, y_train, y_test = prepare_data_for_model(data_rs)

In [6]:
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = diags([1, -2, 1], [0, -1, -2], shape=(L-2, L))
    w = np.ones(L)
    for i in range(niter):
        Z = eye(L) + lam * D.T @ D
        W = spsolve(Z, w * y)
        w = p * (y > W) + (1 - p) * (y <= W)
    return W

# Assuming baseline_als is defined as before

def process_row(row, lam=10**5, p=0.001, window_size=51, poly_order=3, peak_height=0.01):
    y = np.array(row["MALDI_binned"])
    y_filtered = savgol_filter(y, window_size, poly_order)
    y_base_corrected = y_filtered - baseline_als(y_filtered, lam, p)
    y_normalized = y_base_corrected / np.sum(y_base_corrected)
    peaks, _ = find_peaks(y_normalized, height=peak_height)
    return pd.Series([y_normalized.tolist(), peaks.tolist()], index=["processed_MALDI", "peaks"])

def process_samples(data):
    # Ensure the DataFrame has the necessary columns and types
    if 'processed_MALDI' not in data.columns or 'peaks' not in data.columns:
        data = data.assign(processed_MALDI=None, peaks=None)
        data[['processed_MALDI', 'peaks']] = data[['processed_MALDI', 'peaks']].astype('object')
    
    results = data.apply(process_row, axis=1)
    data[['processed_MALDI', 'peaks']] = results
    
    return data


train_data = process_samples(train_data)
test_data = process_samples(test_data)

In [ ]:
X_train = train_data.drop(columns=['Erythromycin', 'Ciprofloxacin', 'y', 'MALDI_binned'])
y_train = train_data['y']

X_test = test_data.drop(columns=['Erythromycin', 'Ciprofloxacin', 'y', 'MALDI_binned'])
y_test = test_data['y']

In [ ]:
# oversample the train data

def expand_lists(data, list_column_names):
    # For each column that contains lists, expand into separate columns
    for col_name in list_column_names:
        temp_df = data[col_name].apply(pd.Series)
        max_len = temp_df.shape[1]
        new_col_names = [f"{col_name}_{i+1}" for i in range(max_len)]
        temp_df.columns = new_col_names
        data = data.drop(col_name, axis=1)
        data = pd.concat([data, temp_df], axis=1)
    
    return data

list_column_names = ['processed_MALDI', 'peaks']
X_train = expand_lists(X_train, list_column_names)
X_test = expand_lists(X_test, list_column_names)

print(X_train.columns)
print(X_test.columns)

## Reducción de dimensionalidad
- PCA
- HSIC Lasso

In [ ]:
# from sklearn.decomposition import PCA

# def pca_feature_selection(x_train, variance_ratio=0.95):
#     pca = PCA(n_components=variance_ratio)
#     pca.fit(x_train)
#     x_train_transformed = pca.transform(x_train)
    
#     return x_train_transformed, pca

# def apply_pca_to_test(pca, x_test):
#     return pca.transform(x_test)


# X_train_pca, pca_model = pca_feature_selection(X_train.to_numpy(), variance_ratio=0.95)
# X_test_pca = apply_pca_to_test(pca_model, X_test.to_numpy())

In [ ]:
def hsic_lasso_feature_selection_and_lr(x_train, y_train, num_features=200):
    hsic_lasso = HSICLasso()
    hsic_lasso.input(x_train, y_train, featname=["mass_" + str(i) for i in range(x_train.shape[1])])
    hsic_lasso.regression(num_feat=num_features, M=5, n_jobs=-1)
    selected_features = hsic_lasso.get_index()
    hsic_lasso.dump()
    selected_features = np.array(selected_features, dtype=int)
    print("Selected features by HSIC-LASSO:", selected_features)

    return selected_features

num_features = X_train.shape[1]  # Number of features/columns in your dataset
x_masses = np.arange(num_features)

selected_features = hsic_lasso_feature_selection_and_lr(X_train.to_numpy(), y_train.to_numpy(), num_features=500)

In [ ]:
def plot_selected_maldi_tofs_stem(x_masses, x_intensities, y_labels, selected_features, method='LASSO'):
    """
    Plots mean±std MALDI-TOF spectra for different labels on the same figure,
    using a stem plot for selected features. Saves the plot in the 'plots' directory.

    Parameters:
    x_masses (numpy.ndarray): Array of mass-to-charge ratios.
    x_intensities (numpy.ndarray): Array of intensity values.
    y_labels (numpy.ndarray): Array of labels.
    selected_features (list or array): Indices of selected features.
    method (str): The feature selection method used ('LASSO' or 'HSIC-LASSO').
    """
    plt.figure(figsize=(10, 6))

    def plot_mean_std_spectrum_stem(label, label_value, color, label_name):
        indices = np.where(y_labels == label_value)[0]
        selected_intensities = x_intensities.iloc[indices, selected_features.tolist()]
        mean_spectrum = np.mean(selected_intensities, axis=0)
        std_spectrum = np.std(selected_intensities, axis=0)
        selected_masses = x_masses[selected_features]

        markerline, stemlines, baseline = plt.stem(
            selected_masses,
            mean_spectrum,
            label=f"Mean - {label_name}",
            linefmt=color,
            basefmt=" ",
        )
        plt.setp(stemlines, "color", color, "linewidth", 1.5)
        plt.setp(markerline, "color", color, "markersize", 4)

    plot_mean_std_spectrum_stem(y_labels, 0, "blue", "None")
    plot_mean_std_spectrum_stem(y_labels, 1, "green", "Ciprofloxacin")
    plot_mean_std_spectrum_stem(y_labels, 2, "red", "Erythromycin")
    plot_mean_std_spectrum_stem(y_labels, 3, "purple", "Both")
    
    plt.title(f"Selected Features MALDI-TOF Spectra by {method} (Stem Plot)")
    plt.xlabel("Selected Mass-to-Charge (m/z)")
    plt.ylabel("Intensity")
    plt.legend()

    # Ensure the 'plots' directory exists
    if not os.path.exists('plots'):
        os.makedirs('plots')

    # Save the plot
    plt.savefig(f'plots/selected_features_{method.lower()}.png')
    wandb.log({"selected_features": wandb.Image(f'plots/selected_features_{method.lower()}.png')})
    plt.close()  # Close the plot to avoid displaying it inline if not desired

plot_selected_maldi_tofs_stem(X_train.columns, X_train, y_train, selected_features, method='HSIC-LASSO')

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, RandomOverSampler
from collections import defaultdict

# Assuming X_train and y_train are your training data and labels

oversampling_methods = {
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'BorderlineSMOTE': BorderlineSMOTE(random_state=42),
    'SVMSMOTE': SVMSMOTE(random_state=42),
    'RandomOverSampler': RandomOverSampler(random_state=42)
}

# Precompute oversampled datasets
oversampled_datasets = defaultdict(dict)
for name, oversampler in oversampling_methods.items():
    X_res, y_res = oversampler.fit_resample(X_train, y_train)
    oversampled_datasets[name]['X'] = X_res
    oversampled_datasets[name]['y'] = y_res

# oversampled_datasets_pca = defaultdict(dict)
# for name, dataset in oversampled_datasets.items():
#     X_res_pca, y_res_pca = oversampler.fit_resample(X_train_pca, y_train)
#     oversampled_datasets_pca[name]['X'] = X_res_pca
#     oversampled_datasets_pca[name]['y'] = dataset['y']

In [ ]:
# # pickle all preprocessed data
# import pickle

# with open('preprocessed_data.pkl', 'wb') as f:
#     pickle.dump([oversampled_datasets, oversampled_datasets_pca, pca_model, selected_features, X_train, y_train, X_test, y_test, train_data, test_data], f)

# # load preprocessed data
# with open('preprocessed_data.pkl', 'rb') as f:
#     oversampled_datasets, oversampled_datasets_pca, pca_model, selected_features, X_train, y_train, X_test, y_test, train_data, test_data = pickle.load(f)

## Modelos

In [ ]:
# Define sweep configuration
sweep_config = {
    'method': 'random',  # Use the random search strategy
    'metric': {
        'name': 'test_f1_score',
        'goal': 'maximize'
    },
    'parameters': {
        'dimensionality_reduction': {
            'values': ['HSIC-Lasso']
        },
        'model': {
            'values': ['LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting', 'CatBoost', 'XGBoost', 'LightGBM']
        },
        'oversampling_method': {
            'values': ['SMOTE', 'ADASYN', 'BorderlineSMOTE', 'SVMSMOTE', 'RandomOverSampler']
        },
        'num_features': {
            'min': 1,
            'max': len(selected_features)
        }
    }
}

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels, title):
    """
    Plots a confusion matrix as a heatmap.

    Parameters:
    y_true (numpy.ndarray): True labels.
    y_pred (numpy.ndarray): Predicted labels.
    labels (list): List of class labels.
    title (str): Title of the plot.
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels).plot(cmap='Blues', ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    
    # Log the plot directly to wandb
    wandb.log({"Confusion Matrix": wandb.Image(fig)})
    
    # Close the plot to free up memory
    plt.close(fig)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x107921550>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x107921550>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
def model_train_evaluate(config=None):
    with wandb.init(config=config):
        config = wandb.config

        with open('preprocessed_data.pkl', 'rb') as f:
            # Load your data
            data = pickle.load(f)
            oversampled_datasets, oversampled_datasets_pca, pca_model, selected_features, X_train, y_train, X_test, y_test, train_data, test_data = data

        # Select features based on config
        if config['dimensionality_reduction'] == 'PCA':
            X_train = oversampled_datasets_pca[config.oversampling_method]['X']
            y_train = oversampled_datasets_pca[config.oversampling_method]['y']
        else:
            X_train = oversampled_datasets[config.oversampling_method]['X']
            y_train = oversampled_datasets[config.oversampling_method]['y']
            subsample_features = selected_features[:config.num_features]
            selected_columns = [f'processed_MALDI_{num}' for num in subsample_features]
            X_train = X_train[selected_columns]
            X_test = X_test[selected_columns]
        
        # Define models
        models = {
            'LogisticRegression': LogisticRegression(max_iter=1000),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'GradientBoosting': GradientBoostingClassifier(),
            'LightGBM': LGBMClassifier(),
            'XGBoost': XGBClassifier(),
            'CatBoost': CatBoostClassifier()
        }

        model = models[config.model]

        # Cross-validation
        cv = StratifiedKFold(n_splits=5)
        scoring = {
            'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score, average='weighted'),
            'recall': make_scorer(recall_score, average='weighted'),
            'f1_weighted': make_scorer(f1_score, average='weighted'),
            'balanced_accuracy': make_scorer(balanced_accuracy_score)
        }
        results = cross_validate(model, X_train, y_train, cv=cv, scoring=scoring)

        # Train the model on the entire training set
        model.fit(X_train, y_train)

        # Evaluate on the test set
        y_pred = model.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred, average='weighted')
        test_recall = recall_score(y_test, y_pred, average='weighted')
        test_f1_score = f1_score(y_test, y_pred, average='weighted')
        test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

        # Log cross-validation results
        wandb.log({
            'cv_f1-score': results['test_f1_weighted'].mean(),
            'cv_accuracy': results['test_accuracy'].mean(),
            'cv_precision': results['test_precision'].mean(),
            'cv_recall': results['test_recall'].mean(),
            'cv_balanced_accuracy': results['test_balanced_accuracy'].mean(),
            # Log test set results
            'test_accuracy': test_accuracy,
            'test_precision': test_precision,
            'test_recall': test_recall,
            'test_f1_score': test_f1_score,
            'test_balanced_accuracy': test_balanced_accuracy
        })

        # Log confusion matrix
        plot_confusion_matrix(y_test, y_pred, ['None', 'Ciprofloxacin', 'Erythromycin', 'Both'], 'Test Set Confusion Matrix')

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x107921550>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x107921550>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="practica_maldi")
wandb.agent(sweep_id, model_train_evaluate, count=100)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x107921550>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Create sweep with ID: ffg302oq
Sweep URL: https://wandb.ai/practica_maldi/practica_maldi/sweeps/ffg302oq
